In [16]:
from torch import nn  # 导入神经网络模块
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np


class Mnist_NN(nn.Module):
    def __init__(self):
        """
        准备网络需要的原材料
        """
        super().__init__()
        self.hidden1 = nn.Linear(784, 128)
        self.hidden2 = nn.Linear(128, 256)
        # self.hidden3 = nn.Linear(256,512)
        self.out = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        """
        构造前向传播网络, 方向传播的网络由框架自动生成
        """
        x = x.view(-1, 784)

        x = F.relu(self.hidden1(x))
        x = self.dropout(x)

        x = F.relu(self.hidden2(x))
        x = self.dropout(x)

        x = self.out(x)

        return x


def get_data(trainset, validset, bs=64):
    return(
        DataLoader(trainset, batch_size=bs, shuffle=True),
        DataLoader(validset, batch_size=bs)
    )


def fit(epochs, model, loss_func, opt, train_dl, vaild_dl):
    for epoch in range(epochs):
        model.train()  # 更新权重和参数
        for x_train, y_train in train_dl:
            loss_batch(model, loss_func, x_train, y_train, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, x, y) for x, y in vaild_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        print("当前epoch:" + str(epoch), "验证集损失:" + str(val_loss))


def loss_batch(model, loss_func, x_train, y_train, opt=None):
    # print(x_train.dtype(), y_train.dtype())
    loss = loss_func(model(x_train), y_train)
    if opt is not None:
        loss.backward()  # 传播方向
        opt.step()  # 更新参数
        opt.zero_grad()  # 清空上一次更新的梯度为0

    return loss.item(), len(x_train)





if __name__ == "__main__":
    # 定义数据预处理， 将多个变换组合成一个变化序列
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5,), (0.5,))])
    # 加载训练数据集
    train_set = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           download=True,
                                           transform=transform)

    # 加载验证数据集
    valid_set = torchvision.datasets.MNIST(root='./data',
                                           train=False,
                                           download=True,
                                           transform=transform)
    train_dl, valid_dl = get_data(train_set, valid_set)
    print("loading data done!")

    model = Mnist_NN()
    opt = torch.optim.SGD(model.parameters(), lr=0.001)

    fit(2, model, F.cross_entropy, opt, train_dl, valid_dl)

loading data done!
当前epoch:0 验证集损失:0.23019367613792419
当前epoch:1 验证集损失:0.16789367280453443
当前epoch:2 验证集损失:0.13968519426696002
当前epoch:3 验证集损失:0.11025581549070776
当前epoch:4 验证集损失:0.11348850785214454
当前epoch:5 验证集损失:0.0994592545909807
当前epoch:6 验证集损失:0.10324749231860042
当前epoch:7 验证集损失:0.10202335448153317
当前epoch:8 验证集损失:0.0909839156942442
当前epoch:9 验证集损失:0.08714337632898242
当前epoch:10 验证集损失:0.10316480679679663
当前epoch:11 验证集损失:0.08489993658084423
当前epoch:12 验证集损失:0.09310972396200523
当前epoch:13 验证集损失:0.0867012240136508
当前epoch:14 验证集损失:0.08868460178859532
当前epoch:15 验证集损失:0.08714220064021647
当前epoch:16 验证集损失:0.07921220764322207
当前epoch:17 验证集损失:0.08017392713977024
当前epoch:18 验证集损失:0.0870297177521512
当前epoch:19 验证集损失:0.07887627218859852


## 计算训练模型的准确率

In [17]:
correct = 0
total = 0
print("I ok")
for x, y in valid_dl:
    print(x)
    pred = model(x)
    _, predicted =  torch.max(pred.data, dim=1) # 返回最大值和最大值的索引
    print(predicted)


torch.return_types.max(
values=tensor([10.4029, 11.1362,  8.1540,  8.6337,  9.0510, 11.3364,  8.7575,  5.7518,
         8.5096,  9.1773, 13.7991,  8.3378, 11.0095, 11.6109, 12.4788,  4.7635,
         7.8654, 10.0419,  4.2958, 11.1802,  8.3812,  9.7368,  8.3109, 16.8603,
         9.6300, 16.4471, 11.0485, 11.5176, 15.8290,  9.0066, 13.1749,  5.1096,
        14.0867,  4.2549, 12.5262,  8.5653,  8.2061, 11.1133,  5.5055, 10.2003,
         8.3920,  9.1028, 11.1305,  2.4319,  8.6716, 10.0483,  6.8108,  7.3633,
        12.0932,  9.8844, 10.3851, 11.0828,  9.6660,  6.8341,  9.7754, 10.1720,
        11.4394, 10.6274, 11.2287,  9.7734, 12.1540,  7.0804,  5.2364,  7.1220],
       grad_fn=<MaxBackward0>),
indices=tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3]))


I ok
准确率为:0.9747
